<a href="https://colab.research.google.com/github/kkattt51/ArtInt/blob/main/5.%22SQL_Tutorial_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL - язык структурированных запросов - практика - Иван Рычков

---

# Знакомство с базой данных

## Реляционная модель данных
- Данные организованы таким образом, чтобы у нас была возможность соединять таблицы между собой по общим полям и брать необходимую информацию из нескольких таблицу сразу.

- Это полезно по той причине, что мы экономим место, записывая данные разово в отдельных справочниках, а не повторяя одинаковые значения много раз.

Наша база описывает музыкальную стриминговую площадку (частично взята с реального iTunes). Каждая из 11 таблиц описывает свой тип данных:
- `employees` - сотрудники и их подчинение.
- `customers` - клиенты.
- `invoices` - операции покупки (квитанции об оплате) с адресами.
- `invoice_items` - детализованные данные о покупках (каждый трек).
- `artists` - исполнители.
- `albums` - альбомы.
- `media_types` - типы медиафайлов.
- `genres` - музыкальные стили.
- `tracks` - композиции (треки).
- `playlists` - списки воспроизведения (музыкальные подборки).
- `playlist_track` - треки, входящие в каждый плейлист.

---

Таблицы в нашей учебной реляционной базе выглядят следующим образом:
![img](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

---

## Подготовка к работе
Перед началом работы мы:
- Установим `toads` для простой работы с SQLite-базами. [Видео по теме](https://youtu.be/27uSAMPU66E).
- Скачаем базу данных с [sqlitetutorial.net](https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip) и распакуем её в папку с ноутбуком.

In [ ]:
# Установка toads
!pip install --upgrade git+https://github.com/ivanrychkov/toads

  Cloning https://github.com/ivanrychkov/toads to /tmp/pip-req-build-1rsuxm1k
  Running command git clone -q https://github.com/ivanrychkov/toads /tmp/pip-req-build-1rsuxm1k
  Created wheel for toads-ivanrychkov: filename=toads_ivanrychkov-0.0.16-cp37-none-any.whl size=11568 sha256=c3d7e85136f2cf524b4e125698ec668dffb7d6fa015e8d87e44a5b619309130a
  Stored in directory: /tmp/pip-ephem-wheel-cache-i97zprd9/wheels/0d/eb/dc/0381896baddafc7c4255ae39e6c479dc9dccdefa8d96a6bd48
Successfully built toads-ivanrychkov


In [ ]:
from toads.sql import SQLocal
import requests
from zipfile import ZipFile, ZipExtFile
import pandas as pd
import os
import re

### Скачивание базы данных
Наша база данных сжата в zip, поэтому нам нужно будет её не только загрузить, но и распаковать.

In [ ]:
# Ссылка на загрузку
download_link = 'https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'

# Извлекаем окончание ссылки как имя файла с помощью регулярного выражения
zip_name = re.search(r'/(\w+\.\w+)$', download_link).group(1)
print('downloading', zip_name + '...')

# Скачиваем zip-папку с базой данных в соответствующий файл
with open(zip_name, 'wb') as file:
  r = requests.get(download_link)
  file.write(r.content)
print('downloaded')

# Распаковываем zip-архив в папку с блокнотом с помощью средства zipfile
ZipFile(zip_name).extractall()
print('file extracted')

# Удаляем zip-папку за ненадобностью
os.remove(zip_name)
print(zip_name, 'removed')

downloading chinook.zip...
downloaded
file extracted
chinook.zip removed


## Подключение к базе данных

In [ ]:
# Создадим объект, который будет смотреть на файл нашей базы данных
sql = SQLocal('chinook.db')

# Перечислим имеющиеся таблицы
sql.list_tables()

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoices',
 'invoice_items',
 'media_types',
 'playlists',
 'playlist_track',
 'tracks']

Можем приступить к написанию запросов.

*Также можно делать запросы здесь через графический интерфейс: https://www.sqlitetutorial.net/tryit/.*

---

# Основные операторы SQL

## SELECT - выбор данных
- Позволяет выбирать любые значения и выводить их как столбцы.
- Выполняется в последнюю очередь.
- \*Также будем использовать alias-ы для столбцов (собственные названия) с помощью оператора `AS`.

In [ ]:
# Выберем просто единицу
sql('''select 1''')

,1
0,1


In [ ]:
# SQL поддерживает арифметические операции
sql('''
select
5 + 10 as addition, -- сложение
5 - 10 as subtraction, --вычитание
5 * 10 as multiplication, --умножение
5 / 10 as division, --деление
--power(2, 5) as power, --возведение двойки в 5 степень. В колабе не работает, но работает в тренажёре.
36 % 24 as modulo --остаток от деления
''')

,addition,subtraction,multiplication,division,modulo
0,15,-5,50,0,12


In [ ]:
# NULL - пропуск в данных
sql('select NULL')

,NULL
0,None


## FROM - выбор данных из таблицы
- Сначала выполняется блок `FROM` - выбирается таблица.
- Затем из таблицы выбираются значения из блока `SELECT`.
- \*Также `LIMIT N` - вывод первых N строк. Пишется в конце запроса.
- \*`SELECT DISTINCT` - вывод уникальных значений/сочетаний значений.

In [ ]:
# * - Выбрать все объекты
sql('select * from invoices')

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...
407,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,409,29,2013-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,410,35,2013-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
410,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


In [ ]:
# Выбор отдельных полей таблицы (10 первых).
sql('''
select name, composer, unitprice
from tracks
limit 10
''')

,Name,Composer,UnitPrice
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",0.99
1,Balls to the Wall,None,0.99
2,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",0.99
3,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",0.99
4,Princess of the Dawn,Deaffy & R.A. Smith-Diesel,0.99
5,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",0.99
6,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",0.99
7,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",0.99
8,Snowballed,"Angus Young, Malcolm Young, Brian Johnson",0.99
9,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",0.99


In [ ]:
# Какие вообще цены существуют в нашем магазине?
sql('''
select distinct unitprice
from tracks
''')

,UnitPrice
0,0.99
1,1.99


## ORDER BY - сортировка строк
- `ORDER BY столбец1 [DESC][, столбец2 [DESC] ...]` - по возрастанию значений в столбце [или убыванию]. Если сортировка по нескольким критериям, то для каждого можно указать, по возрастанию или убыванию.
- Выполняется перед `SELECT`.
- Пишется в конце запроса, но не после `LIMIT`.

In [ ]:
# 3 самых молодых сотрудника (Сортировка по убыванию даты рождения)
sql('''
select firstname, lastname, birthdate
from employees
order by birthdate desc
limit 3
''')

,FirstName,LastName,BirthDate
0,Jane,Peacock,1973-08-29 00:00:00
1,Michael,Mitchell,1973-07-01 00:00:00
2,Robert,King,1970-05-29 00:00:00


In [ ]:
# Сортировка по убыванию даты рождения, по алфавиту фамилии и имени
sql('''
select lastname, firstname, birthdate from employees
order by birthdate desc, lastname, firstname
''')

,LastName,FirstName,BirthDate
0,Peacock,Jane,1973-08-29 00:00:00
1,Mitchell,Michael,1973-07-01 00:00:00
2,King,Robert,1970-05-29 00:00:00
3,Callahan,Laura,1968-01-09 00:00:00
4,Johnson,Steve,1965-03-03 00:00:00
5,Adams,Andrew,1962-02-18 00:00:00
6,Edwards,Nancy,1958-12-08 00:00:00
7,Park,Margaret,1947-09-19 00:00:00


In [ ]:
# Топ-5 покупок по стоимости (сортировка по убыванию)
sql('''
select *
from invoices
order by total desc, invoicedate
limit 5''')

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,404,6,2013-11-13 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,25.86
1,299,26,2012-08-05 00:00:00,2211 W Berry Street,Fort Worth,TX,USA,76110,23.86
2,96,45,2010-02-18 00:00:00,Erzsébet krt. 58.,Budapest,None,Hungary,H-1073,21.86
3,194,46,2011-04-28 00:00:00,3 Chatham Street,Dublin,Dublin,Ireland,None,21.86
4,89,7,2010-01-18 00:00:00,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,18.86


## WHERE - Фильтрация строк по условию
- Оставляет только те строки (срез из таблицы), для которых выполняется заданное нами условие.
- Выполняется после `FROM`.
- Поддерживает несколько условий (операторы `AND` и `OR`).

In [ ]:
# Только покупки из США
sql('''
select invoiceid, invoicedate, total, billingcity, billingcountry
from invoices
where billingcountry = 'USA'
''')

,InvoiceId,InvoiceDate,Total,BillingCity,BillingCountry
0,5,2009-01-11 00:00:00,13.86,Boston,USA
1,13,2009-02-19 00:00:00,0.99,Mountain View,USA
2,14,2009-03-04 00:00:00,1.98,Redmond,USA
3,15,2009-03-04 00:00:00,1.98,Cupertino,USA
4,16,2009-03-05 00:00:00,3.96,Reno,USA
...,...,...,...,...,...
86,397,2013-10-13 00:00:00,13.86,Tucson,USA
87,405,2013-11-21 00:00:00,0.99,Mountain View,USA
88,406,2013-12-04 00:00:00,1.98,Reno,USA
89,407,2013-12-04 00:00:00,1.98,Boston,USA


In [ ]:
# Топ-10 покупок дороже 10$ и не из США
sql('''
select invoiceid, invoicedate, total, billingcity, billingcountry
from invoices
where total >= 10
and billingcountry != 'USA'
order by total desc
limit 10
''')

,InvoiceId,InvoiceDate,Total,BillingCity,BillingCountry
0,404,2013-11-13 00:00:00,25.86,Prague,Czech Republic
1,96,2010-02-18 00:00:00,21.86,Budapest,Hungary
2,194,2011-04-28 00:00:00,21.86,Dublin,Ireland
3,89,2010-01-18 00:00:00,18.86,Vienne,Austria
4,88,2010-01-13 00:00:00,17.91,Santiago,Chile
5,306,2012-09-05 00:00:00,16.86,Prague,Czech Republic
6,313,2012-10-06 00:00:00,16.86,Dijon,France
7,208,2011-06-29 00:00:00,15.86,Oslo,Norway
8,193,2011-04-23 00:00:00,14.91,Frankfurt,Germany
9,12,2009-02-11 00:00:00,13.86,Stuttgart,Germany


In [ ]:
# Страны, в которых есть штаты
sql('''
select distinct
billingcountry,
billingstate
from invoices
where billingstate is not null
order by billingcountry, billingstate
''')

,BillingCountry,BillingState
0,Australia,NSW
1,Brazil,DF
2,Brazil,RJ
3,Brazil,SP
4,Canada,AB
5,Canada,BC
6,Canada,MB
7,Canada,NS
8,Canada,NT
9,Canada,ON


## CASE - присвоение значений по условию
- Позволяет выбирать значения в зависимости от условия, которое выполняется.
- `CASE WHEN условие1 THEN значение1 [WHEN условие2 THEN значение2] [ELSE значение_для_остальных_случаев] END`.

In [ ]:
# Пометим США и остальные страны
sql('''
select
*,
case
    when billingcountry = 'USA' then 1 -- Если США, оставляем как есть
    else 0 -- Иначе указываем, что это не США
    end
    as usa_flag
from invoices
limit 15
''')

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,usa_flag
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,0
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,0
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94,0
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91,0
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86,1
5,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99,0
6,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98,0
7,8,40,2009-02-01 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98,0
8,9,42,2009-02-02 00:00:00,"9, Place Louis Barthou",Bordeaux,None,France,33000,3.96,0
9,10,46,2009-02-03 00:00:00,3 Chatham Street,Dublin,Dublin,Ireland,None,5.94,0


---

# Агрегация данных
- Вычисление статистических показателей над всеми данными в таблице:
  - `SUM()` - сумма
  - `AVG()` - среднее
  - `COUNT()` - количество
  - `MAX()` - максимум
  - `MIN()` - минимум.
- Используется в `SELECT` и при фильтрации значений.

In [ ]:
# Выберем страну с самой дорогой покупкой (MAX)
sql('''
select billingcountry as max_country, max(total) as max_total
from invoices
''')

,max_country,max_total
0,Czech Republic,25.86


In [ ]:
# Средний чек (AVG)
sql('''
select avg(total) as average_total
from invoices
''')

,average_total
0,5.651942


In [ ]:
# Сколько у нас песен в базе? (COUNT)
sql('''
select count(*)
from tracks''')

,count(*)
0,3503


In [ ]:
# В нашем случае это то же самое, что и максимальный id трека (не всегда это так).
sql('''
select max(trackid)
from tracks''')

,max(trackid)
0,3503


In [ ]:
# Сумма всех покупок не менее 5$
sql('''
select
sum(case
    when total >= 5 then total
    else 0
    end) as expensive_invoices_sum
from invoices''')  # Можно было сделать срез через WHERE

,expensive_invoices_sum
0,1797.81


---

# GROUP BY - Группировка данных
- Объекты (строки) в таблицах можно объединять в группы по значениям столбцов, чтобы считать внутри каждой группы её собственные показатели.
- Выполняется и пишется после `WHERE`.
- `HAVING` - аналог `WHERE` для сгруппированных данных. Оставляет только те группы, которые соответствуют условию.

In [ ]:
# Сколько песен в альбомах? (Номер альбома с числом дорожек)
sql('''
select albumid, count(*) tracks
from tracks
group by albumid
order by tracks desc
limit 10''')

,AlbumId,tracks
0,141,57
1,23,34
2,73,30
3,229,26
4,230,25
5,251,25
6,83,24
7,231,24
8,253,24
9,24,23


In [ ]:
# Страны, чей средний чек выше 6$ 
sql('''
select
billingcountry,
avg(total) average_total
from invoices
group by billingcountry
having average_total > 6
order by average_total
''')

,BillingCountry,average_total
0,Austria,6.088571
1,Czech Republic,6.445714
2,Hungary,6.517143
3,Ireland,6.517143
4,Chile,6.660000


In [ ]:
# Суммарное время всех треков по жанрам
sql('''
select genreid,
sum(milliseconds) / 1000 / 3600 as total_hours, -- приводим к секундам и делим на число секунд в часе.
sum(milliseconds) / 60000 % 60 as total_minutes -- Приводим миллисекунды к минутам и берём остаток от деления на 60 минут в часе.
from tracks
group by genreid
order by total_hours desc, total_minutes desc -- Сортировка сначала по часам, затем по минутам в случае с одинаковыми часами
''')

,GenreId,total_hours,total_minutes
0,1,102,17
1,19,55,24
2,21,45,46
3,7,37,27
4,3,32,10
5,4,21,36
6,20,21,1
7,2,10,32
8,18,9,28
9,22,7,29


---

# JOIN - Объединение таблиц
- Таблицы можно присоединять друг к другу по разным принципам, если выполняется условие присоединения (например, значение в столбце А = значению в столбце Б):
    - `[INNER] JOIN` - объединение строк, для которых выполнилось условие (могут потеряться строки исходной таблицы).
    - `LEFT JOIN` - с сохранением всех строк таблицы, к которой мы присоединяем. В строках, для которых не выполнилось условие, будут пропуски.
    - `RIGHT JOIN` - аналогично `LEFT JOIN`, но с сохранением всех столбцов правой таблицы.
    - `FULL JOIN` - с сохранением всех строк обеих таблиц.
- Условие объединения обозначается оператором `ON`: `ON таблица1.столбец1 = таблица2.столбец2`.
    - Можно задавать несколько условий аналогично `WHERE`.
    - \*Если имена столбцов в двух таблицах совпадают, можно использовать `USING(столбец)`.
- Чтобы указать, из какой таблицы мы берём столбец, используется синтаксис с точкой: `таблица.столбец`.
- Выполняется во время `FROM`, то есть в первую очередь.

In [ ]:
# Присоединяем таблицу с жанрами и берём из неё названия жанров
sql('''
select genres.name,
sum(milliseconds) / 1000 / 3600 as total_hours, -- приводим к секундам и делим на число секунд в часе.
sum(milliseconds) / 60000 % 60 as total_minutes -- Приводим миллисекунды к минутам и берём остаток от деления на 60 минут в часе.
from tracks
left join genres
on genres.genreid = tracks.genreid
group by tracks.genreid
order by total_hours desc, total_minutes desc -- Сортировка сначала по часам, затем по минутам в случае с одинаковыми часами
''')

,Name,total_hours,total_minutes
0,Rock,102,17
1,TV Shows,55,24
2,Drama,45,46
3,Latin,37,27
4,Metal,32,10
5,Alternative & Punk,21,36
6,Sci Fi & Fantasy,21,1
7,Jazz,10,32
8,Science Fiction,9,28
9,Comedy,7,29


In [ ]:
sql('''
select *
from tracks
left join media_types
  using(MediaTypeId)
limit 5
''')

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,Name
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,MPEG audio file
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Protected AAC audio file
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,Protected AAC audio file
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Protected AAC audio file
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,Protected AAC audio file


In [ ]:
# У каких групп есть песни в стиле рок? (Промежуточная большая таблица)
sql('''
select *
from tracks t
left join genres as g -- Дадим короткое имя жанрам.
    using(genreid) -- В таблицах одинаково называется столбец, по которому мы соединяем.
left join albums as ab
    using(albumid) -- Добавляем альбомы как промежуточную таблицу, чтобы из неё получить доступ к id исполнителей
left join artists as ar
    on ar.artistid = ab.artistid
where g.name = 'Rock' -- Оставляем только рок (наши таблицы уже объединены)
limit 10
''')

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,Name,Title,ArtistId,ArtistId,Name
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,Rock,For Those About To Rock We Salute You,1,1,AC/DC
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Rock,Balls to the Wall,2,2,Accept
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,Rock,Restless and Wild,2,2,Accept
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Rock,Restless and Wild,2,2,Accept
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,Rock,Restless and Wild,2,2,Accept
5,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,Rock,For Those About To Rock We Salute You,1,1,AC/DC
6,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99,Rock,For Those About To Rock We Salute You,1,1,AC/DC
7,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,Rock,For Those About To Rock We Salute You,1,1,AC/DC
8,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,Rock,For Those About To Rock We Salute You,1,1,AC/DC
9,10,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99,Rock,For Those About To Rock We Salute You,1,1,AC/DC


In [ ]:
# У каких групп есть песни в стиле рок?
sql('''
select distinct
ar.name as artist
from tracks t
left join genres as g -- Дадим короткое имя жанрам.
    using(genreid) -- В таблицах одинаково называется столбец, по которому мы соединяем.
left join albums as ab
    using(albumid) -- Добавляем альбомы как промежуточную таблицу, чтобы из неё получить доступ к id исполнителей
left join artists as ar
    on ar.artistid = ab.artistid
where g.name = 'Rock' -- Оставляем только рок (наши таблицы уже объединены)
order by artist -- Отсортируем по алфавиту
''')

,artist
0,AC/DC
1,Accept
2,Aerosmith
3,Alanis Morissette
4,Alice In Chains
5,Audioslave
6,Creedence Clearwater Revival
7,David Coverdale
8,Deep Purple
9,Def Leppard


In [ ]:
# В каких чеках присутствуют композиции Чака Берри?
# Если совпадений несколько, то к одной строке присоединятся несколько строк
sql('''
select invoiceid
from tracks t
join invoice_items i
on t.trackid = i.trackid
and t.composer = 'Chuck Berry'
''')

,InvoiceId
0,5
1,320


### \*COALESCE - замена пропусков
- `COALESCE(столбец1, [столбец2], значение) AS имя_столбца` - Если в столбце 1 пропуск, смотрим на столбец 2. Если в столбце 2 пропуск, заменяем на оставшееся значение.
- Лаконичнее, чем CASE, но менее гибко, т.к. работает только с пропусками.

In [ ]:
# У нас есть пропуски среди композиторов.
sql('''select name, composer from tracks limit 3''')

,Name,Composer
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson"
1,Balls to the Wall,None
2,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."


In [ ]:
# Там, где не указан композитор, подставим исполнителя.
sql('''
select
t.name,
coalesce(t.composer, ar.name) as composer -- Если не композитор, то исполнитель
from tracks t
left join albums ab
    using(albumid)
left join artists ar
    using(artistid)
limit 3
''')

,Name,composer
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson"
1,Balls to the Wall,Accept
2,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."


---

### JOIN - продолжение: примеры с сотрудниками

In [ ]:
# CROSS JOIN
sql('''
select *
from employees e1
cross join employees e2 -- Каждый сотрудник с каждым
where e1.reportsto = e2.employeeid
''')

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
4,6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
5,7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
6,8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com


In [ ]:
# Кто кому подчиняется? (CROSS JOIN)
sql('''
select e1.firstname || ' ' || e1.lastname as employee, -- Объединяем имя с фамилией через пробел
e1.title as employee_title,
e2.firstname || ' ' || e2.lastname as boss,
e2.title as boss_title
from employees e1
cross join employees e2 -- Каждый сотрудник с каждым
where e1.reportsto = e2.employeeid -- Срез там, где начальник первой таблицы равен id сотрудника из 2 таблицы

/* То же самое более короткой записью
select e1.firstname || ' ' || e1.lastname as employee, -- Объединяем имя с фамилией через пробел
e1.title as employee_title,
e2.firstname || ' ' || e2.lastname as boss,
e2.title as boss_title
from employees e1, employees e2 -- Вместо cross join можно перечислить таблицы через запятую
where e1.reportsto = e2.employeeid -- Срез там, где начальник первой таблицы равен id сотрудника из 2 таблицы.
*/
''')

,employee,employee_title,boss,boss_title
0,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
1,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
2,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
3,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
4,Michael Mitchell,IT Manager,Andrew Adams,General Manager
5,Robert King,IT Staff,Michael Mitchell,IT Manager
6,Laura Callahan,IT Staff,Michael Mitchell,IT Manager


In [ ]:
# Кто кому подчиняется? (LEFT JOIN - не теряем строки там, где не выполнилось условие)
sql('''
select e1.firstname || ' ' || e1.lastname as employee, -- Объединяем имя с фамилией через пробел
e1.title as employee_title,
e2.firstname || ' ' || e2.lastname as boss,
e2.title as boss_title
from employees e1
left join employees e2
    on e1.reportsto = e2.employeeid
''')

,employee,employee_title,boss,boss_title
0,Andrew Adams,General Manager,None,None
1,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
2,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
3,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
5,Michael Mitchell,IT Manager,Andrew Adams,General Manager
6,Robert King,IT Staff,Michael Mitchell,IT Manager
7,Laura Callahan,IT Staff,Michael Mitchell,IT Manager


In [ ]:
# INNER JOIN - если совпадение не нашлось, строки пропадают из исходной таблицы
sql('''
select
e1.firstname || ' ' || e1.lastname as employee, -- Объединяем имя с фамилией через пробел
e1.title as employee_title,
e2.firstname || ' ' || e2.lastname as boss,
e2.title as boss_title
from employees e1
join employees e2
  on e1.reportsto = e2.employeeid
''')
# Видим, что из сотрудников потерялся Andrew Adams, т.к. для него не нашлось совпадения

,employee,employee_title,boss,boss_title
0,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
1,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
2,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
3,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
4,Michael Mitchell,IT Manager,Andrew Adams,General Manager
5,Robert King,IT Staff,Michael Mitchell,IT Manager
6,Laura Callahan,IT Staff,Michael Mitchell,IT Manager


```sql
CREATE TABLE t AS (
  id INTEGER PRIMARY KEY,
  name VARCHAR(24)
)

CREATE TABLE t AS (
  select * from tracks
)

INSERT INTO TABLE t
VALUES (1, 'abc')
```

---

# Домашнее задание:
- Следуя логике задач, вам нужно написать запросы, которые будут возвращать данные в требуемом виде.
- Под каждой задачей будет искомый результат в виде скриншота с таблицей. Как этот результат получить - решать вам.
- После каждой задачи впишите ваше решение, чтобы куратор мог его проверить.
- Всё, что необходимо для решения домашнего задания, можно найти в примерах выше.
- Запросы можно проверить в этой тетрадке и по ссылке: https://www.sqlitetutorial.net/tryit/
- Форма для заполнения находится [здесь](https://docs.google.com/forms/d/e/1FAIpQLScLLSVroT3a1p1D4wi6WQLg4ONuTfNXhx6AobZ6PcD9u1syVw/viewform).
- Задачи [здесь](https://docs.google.com/document/d/1NnQ1Sbp3SSnVTo9wD6rF8c2L99P3uBe89MwvWUdzPQ4/edit).

---